In [1]:
import carla
import sys
import glob
import os
from Utils.Util_functions import distance
sys.path.append("../../carla9.13/PythonAPI/carla/")
# sys.path.append(glob.glob("/Navigation/Utils*"))
# from agents.navigation.behavior_agent import BehaviorAgent
from Controller import VehiclePIDController
# print(client.get_available_maps())
client = carla.Client("localhost",2000)
client.set_timeout(20.0)
world = client.get_world()
print(world)
world = client.load_world('Town06')
vehicle_blueprints = world.get_blueprint_library().filter("cybertruck")
spawn_points = world.get_map().get_spawn_points()
ego_vehicle = world.spawn_actor(vehicle_blueprints[0],spawn_points[25])     #SPWAN POINTS TO BE CONSIDERED  60,61,62,63

spectator = world.get_spectator()
transform = ego_vehicle.get_transform()
spectator.set_transform(carla.Transform(transform.location + carla.Location(x=5,y=0,z=3),
carla.Rotation(pitch=-20,yaw = 180)))

World(id=2549775482705480075)


In [14]:
control = carla.VehicleControl(steer = 0.2)
ego_vehicle.apply_control(control)

In [27]:
a= world.get_map().get_waypoint(ego_vehicle.get_location())
z = [a]
pid = VehiclePIDController(ego_vehicle,{'K_P': 1.95, 'K_I': 0.05, 'K_D': 0.2, 'dt': 0.05},{'K_P': 1.0, 'K_I': 0.05, 'K_D': 0, 'dt': 0.05})
traj = []
counter = 0
try:
    while True:
        control = pid.run_step(20,[-350.651978,-17.373524])
        ego_vehicle.apply_control(control)
        # print(ego_vehicle.get_transform())
        # print(distance([ego_vehicle.get_location().x,ego_vehicle.get_location().y], [593.651978,-17.373524]))
        if distance([ego_vehicle.get_location().x,ego_vehicle.get_location().y], [-350.651978,-17.373524] )< 1:
            ego_vehicle.apply_control(carla.VehicleControl(steer=1.0, throttle = 0.2))
            break
        #     counter += 1
        #     if counter == 4:
        #         z=z[0].get_right_lane().next(60)
        #         z_prev = z[0]
        #     else:
        #         z=z[0].next(10)
        #         z_prev = z[0]
        #     draw_waypoints(world,[z_prev,z[0]])
        # traj.extend([ego_vehicle.get_location()])
        # if counter == 7:..
        #     break
            
except KeyboardInterrupt:
    print("stopped by user, Bye")
    ego_vehicle.destroy
finally:
    ego_vehicle.destroy()

In [13]:
# for i in range(10):
ego_vehicle = world.spawn_actor(vehicle_blueprints[0],spawn_points[30]) 

RuntimeError: Spawn failed because of collision at spawn position

In [ ]:
# trajectory plotting
# x6=[]
# y6=[]
# for i in traj:
#     x6.append(i.x)
#     y6.append(i.y)
import matplotlib.pyplot as plt
fig,axes = plt.subplots(2,3,figsize=(30,15))

axes[0,0].plot(x1,y1)
axes[0,0].axis('equal')
axes[0,0].set_title("10m")

axes[0,1].plot(x2,y2)
axes[0,1].axis('equal')
axes[0,1].set_title("20m")

axes[0,2].plot(x3,y3)
axes[0,2].axis('equal')
axes[0,2].set_title("30m")

axes[1,0].plot(x4,y4)
axes[1,0].axis('equal')
axes[1,0].set_title("40m")

axes[1,1].plot(x5,y5)
axes[1,1].axis('equal')
axes[1,1].set_title("50m")

axes[1,2].plot(x6,y6)
axes[1,2].axis('equal')
axes[1,2].set_title("60m")

fig.tight_layout()
# plt.subplots_adjust(left=0.1,
#                     bottom=0.1, 
#                     right=0.9,
#                     top=0.9, 
#                     wspace=0.4, 
#                     hspace=0.5)
plt.show()
fig.savefig("50m_waypoint_lanechange.jpg")

In [ ]:
agent = BehaviorAgent(ego_vehicle)
agent.set_destination(a.next(1000)[0].transform.location)
traj = []
while True:
    if agent.done():
            print("The target has been reached, stopping the simulation")
            break
    control = agent.run_step()
    control.manual_gear_shift = False
    ego_vehicle.apply_control(control)
    traj.append([ego_vehicle.get_location().x,ego_vehicle.get_location().y])



In [10]:
from Local_planner import Local_planner
from Utils.misc import draw_waypoints
lp = Local_planner(vehicle=ego_vehicle)

# draw_waypoints(world,[carla.Location(X=lp.waypoints_queue[i][0],Y=lp.waypoints_queue[i][1]),carla.Location(x=lp.waypoints_queue[i+1][0],y=lp.waypoints_queue[i+1][1])])

# from Utils.misc import draw_waypoints
# draw_waypoints(world,[vehicle_loc,vehicle_loc.next(1)[0]])
# draw_waypoints(world,[vehicle_loc.get_right_lane(),vehicle_loc.get_right_lane().next(1)[0]])
# draw_waypoints(world,[vehicle_loc.get_left_lane(),vehicle_loc.get_left_lane().next(1)[0]])

ArgumentError: Python argument types in
    Location.__init__(Location)
did not match C++ signature:
    __init__(_object*, carla::geom::Vector3D rhs)
    __init__(_object*, float x=0.0, float y=0.0, float z=0.0)
    __init__(_object*)

In [34]:
map = world.get_map()
print(lp.waypoints_queue)
# for i in range(len(lp.waypoints_queue)-1):
#     draw_waypoints(world,[map.get_waypoint(carla.Location(x=lp.waypoints_queue[i][0],y=lp.waypoints_queue[i][1],z=0))])

deque([[533.6519775390625, -17.373523712158203], [533.3489472545574, -17.37418538815857], [533.0459170064731, -17.374847063014872], [532.7428867938667, -17.375508736750717], [532.4398566157939, -17.376170409389676], [532.1368264713118, -17.37683208095535], [531.8337963594765, -17.37749375147132], [531.5307662793446, -17.378155420961175], [531.2277362299726, -17.37881708944851], [530.9247062104166, -17.379478756956903], [530.6216762197334, -17.38014042350996], [530.3186462569792, -17.38080208913125], [530.0156163212108, -17.381463753844383], [529.7125864114844, -17.38212541767293], [529.4095565268562, -17.382787080640483], [529.1065266663833, -17.383448742770636], [528.8034968291214, -17.38411040408697], [528.5004670141277, -17.38477206461309], [528.197437220458, -17.385433724372568], [527.8944074471692, -17.386095383389005], [527.5913776933176, -17.386757041685982], [527.2883479579593, -17.387418699287082], [526.9853182401514, -17.38808035621591], [526.6822885389497, -17.38874201249604

In [ ]:
ego_vehicle.destroy()

In [ ]:
from collections import deque

A = deque([[1,2],[3,4],[5,6]])
for i in range(len(A)):
    print(A[0])
    A.popleft()
    # print(A)